### קוד מבוא

### ספריות

In [55]:
import os
import sys
import pandas as pd
import geopandas as gpd
import fiona

In [56]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### העלת משתנים להרצת הקוד

In [57]:
path = os.getcwd()

software_root_folder = os.path.dirname(path)

software_folder_location = r'{}\current'.format(software_root_folder)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_folder_location))

forecast_version_folder_location=df_inputs_outputs['location'][1]

sys.path.append(software_root_folder)

### פונקציות

In [58]:
from global_functions import up_load_shp, up_load_df, drop_geo, make_point, up_load_gdb

### פונקציות גלובליות

### העלאת טבלת המרה בין למס מרכזי למשני

In [59]:
stat_join_from_main_to_secondary=up_load_df(
        r'{}\background_files'.format(software_folder_location),'stat_join_from_main_to_secondary')

### העלת טבלת יח_ד למ_ס

In [60]:
folder_path=r'{}\background_files'.format(software_folder_location)
file_name='מספר דירות לפי אזורים סטטיסטיים'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df)
df=df.dropna(how='all')

stat_aprt=df

In [61]:
col=['STAT', 'aprt_20']

stat_aprt=stat_aprt[col]

### שכבת אזורים סטטיסטים

In [62]:
path=r'{}\background_files\statisticalareas_2020_demography.gdb'.format(software_folder_location)
layer_list=fiona.listlayers(path)

layer_name='statisticalareas_2020_demography_jtmt_area'

gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name)).fillna(0)

stat=gpd_layer.rename(columns={'YISHUV_STAT11':'STAT'})

In [63]:
col_name=['STAT','geometry']

stat=stat[col_name]

stat=pd.merge(stat,stat_aprt,on='STAT',how='left').merge(stat_join_from_main_to_secondary,left_on='STAT',right_on='secondary_stat',how='left')

stat.loc[stat['precent_of_stat_data'].isna(),'main_stat']=stat['STAT']

stat.loc[stat['precent_of_stat_data'].isna(),'secondary_stat']=stat['STAT']

stat.loc[stat['precent_of_stat_data'].isna(),'precent_of_stat_data']=1

###  מידע ברמת רשות

In [64]:
stat_point=make_point(stat[['STAT', 'geometry']])

muni_under_JTMT_ITM=up_load_gdb(r'{}\background_files\MUNI_border.gdb'.format(software_folder_location),'muni_under_JTMT_ITM')

muni_under_JTMT_ITM=muni_under_JTMT_ITM[['CR_PNIM','geometry']]

stat=stat.set_index('STAT')

stat['CR_PNIM']=stat_point.sjoin(muni_under_JTMT_ITM)[['STAT','CR_PNIM']].set_index('STAT')

stat=stat.reset_index()

###  העלת מידע אוכלוסייה

In [65]:
folder_path=r'{}\background_files'.format(software_folder_location)
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='classification')
df=df.dropna(how='all')

classification=df

In [66]:
col_name=['main_stat','classification_name']

classification=classification[col_name]

classification=classification.drop_duplicates(subset='main_stat',keep='first')

stat=pd.merge(stat,classification,on='main_stat',how='left')

In [67]:
folder_path=r'{}\background_files'.format(software_folder_location)
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='hh_size')
df=df.dropna(how='all')

hh_size=df

In [68]:
stat=pd.merge(stat,hh_size,on='classification_name',how='left')

In [69]:
folder_path=r'{}\background_files'.format(software_folder_location)
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='types')
df=df.dropna(how='all')

age_des_types=df

age_des_types.pivot_table(columns='age_name',index='classification_name',aggfunc=sum,values='perc').to_excel(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם\התפלגות_גילים_קטלוג\age_des_types_230719.xlsx')

In [70]:
folder_path=r'{}\background_files'.format(software_folder_location)
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='absolute_numbers')
df=df.dropna(how='all').fillna(0)

cbs_pop=df

cbs_pop['pop']=cbs_pop.iloc[:,3:].sum(axis=1)

In [71]:
age=['0','5','10','15','20','25','30','35','40','45','50','55','60','65','70','75','80','85']

for x in age:
    cbs_pop['pop_{}'.format(x)]= cbs_pop['female_{}'.format(x)]+cbs_pop['male_{}'.format(x)]

cbs_pop['pop_75up']= cbs_pop[['pop_75', 'pop_80', 'pop_85']].sum(axis=1)

In [72]:
col=[ 'main_stat', 'pop', 'pop_0', 'pop_5', 'pop_10', 'pop_15', 'pop_20', 'pop_25', 'pop_30', 'pop_35', 'pop_40', 'pop_45', 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

cbs_pop=cbs_pop[col]

יצירת טבלת התפלגות גילים באחוזים לטובת המשך הקוד כאשר יש שינויים בסך הכל אוכלוסיה אבל מעוניינים עדיין בהתפלגות גילים המקורית

In [73]:
cbs_pop_pre=cbs_pop.copy()

col=list(cbs_pop_pre.iloc[:,1:])

cbs_pop_pre.loc[:, col] = cbs_pop_pre.loc[:, col].div(cbs_pop_pre['pop'], axis=0)

cbs_pop_pre=cbs_pop_pre.fillna(0)

cbs_pop_pre=cbs_pop_pre.rename(columns={'pop':'pop_pre'})

cbs_pop_pre=cbs_pop_pre.merge(cbs_pop[['main_stat', 'pop']],on='main_stat',how='left')

stat=pd.merge(stat,cbs_pop_pre,on='main_stat',how='left')

stat['pop']=stat['pop']*stat['precent_of_stat_data']

###  אחוז חרדים מכלל א"ס

In [74]:
folder_path=r'{}\background_files'.format(software_folder_location)
file_name='אוכלוסייה חרדית לפי מחוז, יישוב, ואזור סטטיסטי - 2020'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df)
df=df.dropna(how='all')

stat_hardi=df

In [75]:
stat_hardi=stat_hardi[['pre_hardi','main_stat']].set_index('main_stat')

stat=stat.set_index('main_stat')

stat['pre_hardi']=stat_hardi['pre_hardi']

stat['pre_hardi']=stat['pre_hardi'].fillna(0)

###  תיקון שלב א ערבי ירושלים

In [76]:
path=r'{}\background_files\statisticalareas_2020_demography.gdb'.format(software_folder_location)
layer_name='statisticalareas_2020_demography_arab_jtmt'

stat_arab=up_load_gdb(path,layer_name)

In [ ]:
col=['STAT', 'group_name']

stat_arab=stat_arab[col]

stat_arab=stat_arab.merge(cbs_pop_pre,left_on='STAT',right_on='main_stat',how='left')

stat_arab=stat_arab.set_index('group_name')

group_pop=stat_arab.groupby(by='group_name').sum()[['pop']]

stat_arab['group_pop']=group_pop['pop']

stat_arab['pre_from_group_pop']=stat_arab['pop']/stat_arab['group_pop']

In [ ]:
data = [['akev', 55000], ['Shuafat', 75000],['east_jeru_left',0]]

group_pop_jtmt = pd.DataFrame(data, columns=['group_name', 'pop']).set_index('group_name')

group_pop_delta=group_pop-group_pop_jtmt

group_pop_delta=group_pop_delta.loc['east_jeru_left']+(group_pop_delta.loc['Shuafat']+group_pop_delta.loc['akev'])*0.8#בגלל שהדלתא מוסבר עי הגירה מיוש ולא רק מהעיר ירושלים 

group_pop_jtmt.loc['east_jeru_left','pop']=group_pop_delta.item()

In [ ]:
stat_arab['group_pop_jtmt']=group_pop_jtmt['pop']

stat_arab['pop']=stat_arab['pre_from_group_pop']*stat_arab['group_pop_jtmt']

###  הטמעת תיקון שלב א

In [ ]:
stat_arab=stat_arab.set_index('main_stat')

In [ ]:
stat['pop_cbs']=stat['pop']

stat.loc[list(stat_arab.index),'pop']=stat_arab['pop']

stat['change_from_cbs']=''

stat.loc[list(stat_arab.index),'change_from_cbs']='| general_arab_change |'

###  העלת א"ס בעייתי לתיקון שלב ב

In [ ]:
path=r'{}\background_files\jtmt_fix_for_cbs_data_2020.shp'.format(software_folder_location)

jtmt_fix_stat=up_load_shp(path)

In [ ]:
col=['STAT',
 'fix_pop',
 'fix_aprt',
 'fix_class']

jtmt_fix_stat=jtmt_fix_stat[col].set_index('STAT')

In [ ]:
stat=stat.set_index('STAT')

stat=stat.join(jtmt_fix_stat, how='left')

###  יצירת גמ לפי קטלוג

In [ ]:
cbs_not_lie_lst=list(stat.query('fix_pop.isna() & fix_aprt.isna() & fix_class.isna() ').index)

In [ ]:
stat['count']=1

stat=stat.reset_index()

stat.loc[stat['STAT'].isin(cbs_not_lie_lst),['fix_pop','fix_aprt','fix_class']]=1

In [ ]:
hh_size_by_classification=drop_geo(stat).loc[(stat['aprt_20']>0)&(stat['STAT'].isin(cbs_not_lie_lst))].pivot_table(index='classification_name',aggfunc=sum)[['aprt_20','pop','count']]

hh_size_by_classification['hh_size']=hh_size_by_classification['pop']/hh_size_by_classification['aprt_20']

hh_size_by_classification=hh_size_by_classification[['hh_size','count']]

In [ ]:
hh_size=hh_size.merge(hh_size_by_classification.reset_index(),how='left',on='classification_name',suffixes=('','_cbs'))

###  תיקון שלב ב

In [ ]:
stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==1)&(stat['fix_class']==1),'pop']=stat['aprt_20']*stat['hh_size']

stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==1)&(stat['fix_class']==1),'change_from_cbs']=stat['change_from_cbs']+'| pop_created_from_cbs_aprt |'

stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==0)&(stat['fix_class']==0),'pop']=0

stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==0)&(stat['fix_class']==0),'change_from_cbs']=stat['change_from_cbs']+'| cbs_pop_deleted |'

stat['aprt_20_cbs']=stat['aprt_20']

stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==0)&(stat['fix_class']==0),'aprt_20']=0

stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==0)&(stat['fix_class']==0),'change_from_cbs']=stat['change_from_cbs']+'| cbs_aprt_deleted |'

stat.loc[(stat['fix_pop']==1)&(stat['fix_aprt']==0)&(stat['fix_class']==1),'aprt_20']=stat['pop']/stat['hh_size']

stat.loc[(stat['fix_pop']==1)&(stat['fix_aprt']==0)&(stat['fix_class']==1),'change_from_cbs']=stat['change_from_cbs']+'| aprt_created_from_pop |'

כאן מתקנים מספר דירות לפי גודל משק בית באזורים ערביים שהגודל משק בית יותר או פחות 20% ממה שאנחנו חושבים שצריך להיות שם

In [ ]:
stat.loc[(stat['classification_name'].str.contains("ערבי")==True)&(((stat['pop']/stat['aprt_20'])/stat['hh_size']>1.2)|((stat['pop']/stat['aprt_20'])/stat['hh_size']<0.8)),'aprt_20']=stat['pop']/stat['hh_size']

stat.loc[(stat['classification_name'].str.contains("ערבי")==True)&(((stat['pop']/stat['aprt_20'])/stat['hh_size']>1.2)|((stat['pop']/stat['aprt_20'])/stat['hh_size']<0.8)),'change_from_cbs']=stat['change_from_cbs']+'| aprt_created_from_pop_because_hh_size_not_in_range |'

###  יצירת דירות לפי גודל משק בית

In [ ]:
stat.loc[((stat['aprt_20'].isna())|(stat['aprt_20']==0))&(stat['pop']>0),'change_from_cbs']=stat['change_from_cbs']+'| aprt_created_from_pop_because_no_cbs_data |'

stat.loc[((stat['aprt_20'].isna())|(stat['aprt_20']==0))&(stat['pop']>0),'aprt_20']=round(stat['pop']/stat['hh_size'])

### בקרת מידע למס ברמה של רשות


In [ ]:
pop_2020_cbs_muni=up_load_df(r'{}\background_files'.format(software_folder_location),'pop_2020_cbs_muni')

In [ ]:
stat['CR_PNIM']=stat['CR_PNIM'].fillna(0).astype(int)

stat_by_muni_sum=drop_geo(stat).pivot_table(index='CR_PNIM',aggfunc=sum)[['pop_cbs','pop']]

pop_2020_cbs_muni=pop_2020_cbs_muni.set_index('CR_PNIM')

pop_2020_cbs_muni.join(stat_by_muni_sum,how='inner')

### ייצא מידע ברמת אזור סטט עם השינוים שלנו

In [ ]:
stat=stat.fillna(0)

stat['pop_delta']=stat['pop']-stat['pop_cbs']

stat['aprt_20_delta']=stat['aprt_20']-stat['aprt_20_cbs']

col=['STAT','precent_of_stat_data','classification_name','pop','pop_cbs','aprt_20','aprt_20_cbs','pop_delta','aprt_20_delta','change_from_cbs']

In [ ]:
stat[col].query('STAT!=5526 & STAT!=9975').to_excel(r'{}\Intermediates\stat_cbs_jtmt_2020_short.xlsx'.format(software_folder_location),index=False)

In [ ]:
stat.query('STAT!=5526 & STAT!=9975').to_excel(r'{}\Intermediates\stat_cbs_jtmt_2020.xlsx'.format(software_folder_location),index=False)

path=r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\למ_ס\אזורים סטטיסטים\statisticalareas_2020_demography.gdb'
layer_list=fiona.listlayers(path)

layer_name='statisticalareas_2020_demography_jtmt_area'

gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name)).fillna(0)

stat_border=gpd_layer.rename(columns={'YISHUV_STAT11':'STAT'})

stat_border[['geometry','STAT']].merge(stat[col],on='STAT',how='right').query('STAT!=5526 & STAT!=9975').to_file(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\output\stat_cbs_jtmt_2020.shp',encoding='utf-8')